In [23]:
import pandas as pd
from numpy.ma.extras import column_stack

df = pd.read_csv('resources/historical_hazard_summaries_200-duplicate.csv', encoding='utf-8')

hazard_df = df

cut_off_index = 71

df_truncated = df.iloc[:cut_off_index + 1]

name_list = [
    'Unnamed: 1',
    'A. Total Jurisdictions in BCAT database',
    'B. Jurisdictions in Hazard-prone Regions',
    'C.  Jurisdictions in Hazard-prone Regions that have adopted Commercial Building Codes with Disaster-Resistant Provisions',
    'D.  Jurisdictions in Hazard-prone Regions that have adopted Residential Building Codes with Disaster-Resistant Provisions',
    'E.  Jurisdictions in Hazard-prone Regions that have adopted Commercial and Residential Building Codes with Disaster-Resistant Provisions',
    'E/A Ratio: Jurisdictions with Hazard-Resistant Commercial and Residential Codes to Jurisdictions in BCAT database'
]

df_new = df_truncated.drop(columns=name_list, inplace=True)


print(df_truncated)

              Hazard(s)  \
0             Sep. 2008   
1             Dec. 2008   
2             Mar. 2009   
3             Jun. 2009   
4             Sep. 2009   
..                  ...   
67  Dec. 2024 (FY25 Q1)   
68  Mar. 2025 (FY25 Q2)   
69  Jun. 2025 (FY25 Q3)   
70  Sep. 2025 (FY25 Q4)   
71                  NaN   

   E/B Ratio:\nJurisdictions with Hazard-Resistant Commercial and Residential Codes to Jurisdictions with Hazard-prone Regions  
0                                                 34%                                                                           
1                                                 36%                                                                           
2                                                 37%                                                                           
3                                                 38%                                                                           
4                             

/var/folders/kv/cbwgcq1j0z3b_k5yhl202mh80000gn/T/ipykernel_3189/579900186.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Above I isolated the rows I needed to clean the data, now I will rename the columns and clean the Hazards column to be more uniform.

In [24]:
df_renamed = df_truncated.rename(columns={'E/B Ratio:\nJurisdictions with Hazard-Resistant Commercial and Residential Codes to Jurisdictions with Hazard-prone Regions': 'percentages'})

df_renamed["Hazard(s)"] = df_renamed["Hazard(s)"].replace(r"\s*\(.*?\)", "", regex=True)
df_renamed["percentages"] = df_renamed["percentages"].replace(r"%", "", regex=True)

print(df_renamed)

    Hazard(s) percentages
0   Sep. 2008          34
1   Dec. 2008          36
2   Mar. 2009          37
3   Jun. 2009          38
4   Sep. 2009          39
..        ...         ...
67  Dec. 2024          35
68  Mar. 2025          19
69  Jun. 2025          21
70  Sep. 2025          21
71        NaN         NaN

[72 rows x 2 columns]


Data is cleaned, time to export it to excel - nationwide percentage = y-axis

In [25]:
df_renamed.to_excel('resources/cleaned_historical_hazard_sum2.xlsx', index=False)

Now that we can export it into an excel file, we are going to make it an interactive graph using Plotly!

In [30]:
df_renamed['Hazard(s)'] = pd.to_datetime(df_renamed['Hazard(s)'])
df_sort = df_renamed.sort_values(by='Hazard(s)')

df_renamed["percentages"] = pd.to_numeric(
    df_renamed["percentages"], errors="coerce"
)

In [31]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_sort['Hazard(s)'],
    y=df_sort['percentages'],
    mode='lines',
    name='Quarterly BCAT Percentages'# Name in the legend
))

i = 42 #location for drop - tornadoes
fig.add_annotation(x=df_sort['Hazard(s)'].iloc[i], y=df_sort['percentages'].iloc[i],
            text="BCAT begins tracking tornado hazard",
            font=dict(
                family="Times New Roman",
                color="Black"),
            showarrow=True,
            arrowhead=1,
            arrowcolor="black"),


# Layout parameters
fig.update_layout(
    title=dict(
        text='Building Codes Adoption Status Throughout the Years',
        font=dict(color='Black',
                  family='Times New Roman')       ),  # Title
    xaxis_title='',  # x-axis name
    yaxis_title='Nationwide Percentage',  # y-axis name
    xaxis_tickangle=45,  # Set the x-axis label angle
    showlegend=True,

    paper_bgcolor='white',  # Overall chart background
    plot_bgcolor='white', # Plot area background
    xaxis=dict(
        gridcolor="lightgray",
        griddash="dot",
    ),
    yaxis=dict(
        gridcolor="lightgray",
        griddash="dot",
        ticksuffix="%"
    ),
    font=dict(family="Times New Roman", color="black"),
    legend=dict(yanchor='bottom', xanchor='center', y=-0.45, x=0.5, orientation='h')
)

fig.write_html("html_pages/line_graph.html")


In [32]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_sort['Hazard(s)'],
    y=df_sort['percentages'],
    mode='lines',
    fill='tozeroy',
    fillcolor='rgba(146, 152, 240, 0.3)',
    name='Quarterly BCAT Percentages'# Name in the legend
))

i = 42 #location for drop - tornadoes
fig.add_annotation(
    x=df_sort['Hazard(s)'].iloc[i],
    y=df_sort['percentages'].iloc[i],
    text="BCAT begins tracking tornado hazard",
    showarrow=True,
    arrowhead=1,
    arrowcolor="black"),

# Layout parameters
fig.update_layout(
    title=dict(
        text='Building Codes Adoption Status Throughout the Years',
        font=dict(color='Black',
                  family='Times New Roman')       ),  # Title
    xaxis_title='',  # x-axis name
    yaxis_title='Nationwide Percentage',  # y-axis name
    xaxis_tickangle=45,  # Set the x-axis label angle
    showlegend=True,

    paper_bgcolor='white',  # Overall chart background
    plot_bgcolor='white', # Plot area background
    xaxis=dict(
        gridcolor="lightgray",
        griddash="dot",
    ),
    yaxis=dict(
        gridcolor="lightgray",
        griddash="dot",
        ticksuffix="%"
    ),
    font=dict(family="Times New Roman", color="black"),
    legend=dict(yanchor='bottom', xanchor='center', y=-0.45, x=0.5, orientation='h')
)

fig.write_html("html_pages/area_plot.html")


In [33]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_sort['Hazard(s)'],
    y=df_sort['percentages'],
    name='Quarterly BCAT Percentages',
))

i = 42 #location for drop - tornadoes
fig.add_annotation(
    x=df_sort['Hazard(s)'].iloc[i],
    y=df_sort['percentages'].iloc[i],
    text="BCAT begins tracking tornado hazard",
    showarrow=True,
    arrowhead=1,
    arrowcolor="black"),

# Layout parameters
fig.update_layout(
    title=dict(
        text='Building Codes Adoption Status Throughout the Years',
        font=dict(color='Black',
                  family='Times New Roman')       ),  # Title
    xaxis_title='',  # x-axis name
    yaxis_title='Nationwide Percentage',  # y-axis name
    xaxis_tickangle=45,  # Set the x-axis label angle
    showlegend=True,

    paper_bgcolor='white',  # Overall chart background
    plot_bgcolor='white', # Plot area background
    xaxis=dict(
        gridcolor="lightgray",
        griddash="dot",
    ),
    yaxis=dict(
        gridcolor="lightgray",
        griddash="dot",
        ticksuffix="%"
    ),
    font=dict(family="Times New Roman", color="black"),
    legend=dict(yanchor='bottom', xanchor='center', y=-0.45, x=0.5, orientation='h')
)

fig.write_html("html_pages/bar_graph.html")
